![](https://fenc.wang/imgs/2020/08/3565654bfbe9ffa0.png)

In [1]:
import tushare as ts
import pandas as pd

#设置完整显示数据
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:,.2f}'.format)

pro = ts.pro_api('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')

def cagr(start_value, end_value, num_periods):
    return (end_value / start_value) ** (1 / (num_periods - 1)) - 1

In [4]:
#时间段
start_date = '20150101'
end_date = '20201231'

#股票池
stock_list = ['爱旭股份', '晶澳科技', '协鑫集成', '中环股份', '东方日升', '中来股份', '亿晶光电', '天合光能', '向日葵', '隆基股份', '通威股份', '正泰电器', '航天机电', '拓日新能']

In [9]:
result_list = []
stock_basic = pro.stock_basic(exchange='', list_status='L', fields='ts_code,symbol,name')
stock_selections = stock_basic[stock_basic['name'].isin(stock_list)]

for ticker, name in zip(stock_selections['ts_code'], stock_selections['name']):
    result_dict = {}
    total_revenue = pro.income(ts_code=ticker, start_date=start_date, end_date=end_date,fields='ts_code,end_date,total_revenue')
    total_revenue.drop_duplicates(subset=['ts_code','end_date'],keep='first',inplace=True)
    total_revenue.set_index('end_date', inplace=True)
    total_revenue_yrs = total_revenue.filter(like='1231', axis=0)
    
    #计算CAGR
    start_value = float(total_revenue_yrs.iloc[-1]['total_revenue'])
    end_value = float(total_revenue_yrs.iloc[0]['total_revenue'])
    num_periods = len(total_revenue_yrs['total_revenue'])
    cagr_y = cagr(start_value, end_value, num_periods)
    
    result_dict['代码'] = ticker
    result_dict['股票名称'] = name
    result_dict['年复合增长率'] = cagr_y * 100
    result_dict['周期'] = num_periods - 1
    result_list.append(result_dict)
    
df = pd.DataFrame(result_list)
df.sort_values(by='年复合增长率', ascending=False)

,代码,股票名称,年复合增长率,周期
10,600732.SH,爱旭股份,106.03,5
2,002459.SZ,晶澳科技,101.91,5
11,601012.SH,隆基股份,54.97,5
6,300393.SZ,中来股份,48.85,5
5,300118.SZ,东方日升,37.30,5
0,002129.SZ,中环股份,28.78,5
3,002506.SZ,协鑫集成,26.33,5
8,600438.SH,通威股份,19.50,5
12,601877.SH,正泰电器,18.81,5
1,002218.SZ,拓日新能,14.07,5


![](https://fenc.wang/imgs/2020/08/3565654bfbe9ffa0.png)